In [1]:
#!pip install selenium

In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

import time
import re
import pickle
import os
import random
import pandas as pd
import numpy as np

In [3]:
# # Make variables for LinkedIn user
# USER_LOGIN = 'stanislav.sigarev@gmail.com'
# USER_PASSWORD = ''

In [4]:
def get_and_save_profile_info(driver, profile_url):
    # This will open the link
    driver.get(profile_url)
    
    # Random sleeping time to load all data
    time.sleep(random.uniform(4, 7))

    # Extracting data from page with BeautifulSoup
    src = driver.page_source

    # Now using beautiful soup    
    soup = BeautifulSoup(src, 'lxml')

    # Extracting the HTML of the complete introduction box
    # that contains the name, status, and the location
    intro = soup.find('div', {'class': 'pv-text-details__left-panel'})
    
    # Extracting the Name
    # In case of an error, try changing the tags used here.
    name_loc = intro.find("h1")
    # strip() is used to remove any extra blank spaces
    name = name_loc.get_text().strip()

    # This gives us the HTML of the tag in which user status is present
    status_loc = intro.find("div", {'class': 'text-body-medium'})
    # Extracting user status
    status = status_loc.get_text().strip()
    
    # Extracting the Company name
    work_space = soup.find('ul', {'class': 'pv-text-details__right-panel'})
    # This gives us the HTML of the tag in which the Company Name is present
    works_at_loc = work_space.find('span', {'class': 'pv-text-details__right-panel-item-text'})
    # Extracting the Company Name
    works_at = works_at_loc.get_text().strip()

    time.sleep(0.5)

    # Get current url from browser
    cur_profile_url = driver.current_url
    
    # Return results for current user
    return [name, status, works_at, cur_profile_url]

    # Print collected data
#     print("Name -->",  name,
#           "\nStatus -->", status,
#           "\nWorks At -->", works_at,
#           "\nProfile URL -->", cur_profile_url)
    
    # Parse posts
#     get_and_print_user_posts(driver, cur_profile_url + POSTS_URL_SUFFIX)    

In [ ]:
# def get_and_print_user_posts(driver, posts_url):
#     driver.get(posts_url)

#     #Simulate scrolling to capture all posts
#     SCROLL_PAUSE_TIME = 1.5

#     # Get scroll height
#     last_height = driver.execute_script("return document.body.scrollHeight")

#     # We can adjust this number to get more posts
#     NUM_SCROLLS = 5

#     for i in range(NUM_SCROLLS):
#         # Scroll down to bottom
#         driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

#         # Wait to load page
#         time.sleep(SCROLL_PAUSE_TIME)

#         # Calculate new scroll height and compare with last scroll height
#         new_height = driver.execute_script("return document.body.scrollHeight")
#         if new_height == last_height:
#             break
#         last_height = new_height

#     # Parsing posts
#     src = driver.page_source

#     # Now using beautiful soup
#     soup = BeautifulSoup(src, 'lxml')
#     # soup.prettify()

#     posts = soup.find_all('li', class_='profile-creator-shared-feed-update__container')
#     # print(posts)

#     print(f'Number of posts: {len(posts)}')
#     for post_src in posts:
#         post_text_div = post_src.find('div', {'class': 'feed-shared-update-v2__description-wrapper mr2'})

#         # if post_text_div is None:
#         #     print(post_src)

#         if post_text_div is not None:
#             post_text = post_text_div.find('span', {'dir': 'ltr'})
#         else:
#             post_text = None

#         # If post text is found
#         if post_text is not None:
#             post_text = post_text.get_text().strip()
#             print(f'Post text: {post_text}')

#         reaction_cnt = post_src.find('span', {'class': 'social-details-social-counts__reactions-count'})

#         # If number of reactions is written as text
#         # It has different class name
#         if reaction_cnt is None:
#             reaction_cnt = post_src.find('span', {'class': 'social-details-social-counts__social-proof-text'})

#         if reaction_cnt is not None:
#             reaction_cnt = reaction_cnt.get_text().strip()
#             print(f'Reactions: {reaction_cnt}')

#     return

In [5]:
# Количество страниц результатов поиска для парсинга
NUM_PAGES_TO_PARSE = 2

# Название файла cookies
COOKIES_PATH = '\lincookies'

# Папка для хранения файлов
FILES_PATH = 'C:\Recruiters'

# Путь к файлу с URL
URL_FILE_NAME = FILES_PATH + r"\urls.csv"

# Путь к файлу с инфо пользователей
INFO_FILE_NAME = FILES_PATH + r"\user-info.csv"

# Суффикс ссылки на посты пользователя
POSTS_URL_SUFFIX = 'recent-activity/all/'

In [6]:
# Проверка наличия папки для cookies
if not os.path.exists(FILES_PATH):
    os.mkdir(FILES_PATH)

if __name__ == '__main__':
    caps = DesiredCapabilities().CHROME
    caps['pageLoadStrategy'] = 'eager'
    driver = webdriver.Chrome()
    # Установлен широкоформатный размер экрана, чтобы нужные элементы помещались на экране
    driver.set_window_size(1920, 1080)

    # Авторизация по cookies
    if os.path.exists(FILES_PATH + COOKIES_PATH):
        driver.get('https://linkedin.com')
        #Загрузка куки
        for cookie in pickle.load(open(FILES_PATH + COOKIES_PATH, 'rb')):
            driver.add_cookie(cookie)
        time.sleep(5)
        driver.refresh()
        time.sleep(10)

    # Вход по паролю и сохранение cookies
    else:
        driver.get('https://linkedin.com/uas/login')
        #Авторизация
        time.sleep(3.5)
        username = driver.find_element(By.ID, "username")
        username.send_keys(str(input('Введите логин: ')))
        pword = driver.find_element(By.ID, "password")
        pword.send_keys(str(input('Введите пароль: ')))
        driver.find_element(By.XPATH, "//button[@type='submit']").click()
        time.sleep(10) #время на ввод кода подтверждения если понадобится
        pickle.dump(driver.get_cookies(), open(FILES_PATH + COOKIES_PATH, 'wb'))

    # Загрузка страницы с результатами поиска
    driver.get('https://www.linkedin.com/search/results/people/?keywords=data%20scientist&origin=CLUSTER_EXPANSION&sid=1gy')
    time.sleep(random.uniform(.5, 3))

    # Необходимые переменные
    profile_urls = []
    profile_info = pd.DataFrame({'name': [],
     'status': [],
     'company': [],
     'profile_url': []})

    for i in range(NUM_PAGES_TO_PARSE):
        # Текущая страница
        print('Page', i+1)
        # Задержка для полной загрузки страницы
        time.sleep(random.uniform(4, 7))

        search_result_links = driver.find_elements(By.CSS_SELECTOR, "span.entity-result__title-text a.app-aware-link")

        for link in search_result_links:
            href = link.get_attribute("href")
            if 'linkedin.com/in' in href:
                profile_urls.append(href)

        last_height = driver.execute_script("return document.body.scrollHeight")
        NUM_SCROLLS = 20

        for j in range(NUM_SCROLLS):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(random.uniform(1.5, 3))
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        time.sleep(random.uniform(4, 7))
                
        print('Sucсess!')
        print('-' * 50)
        
        next_button = driver.find_element(By.CSS_SELECTOR, 'button.artdeco-pagination__button--next')
        next_button.click()
# #Не понял смысла этого цикла, так как в случае исключения не делается ничего, чтобы его устранить
#         k = 1
#         errors = 0
#         while k == 1 and errors < 10:
#             try:
#                 next_button = driver.find_element(By.CSS_SELECTOR, 'button.artdeco-pagination__button--next')
#                 next_button.click()
#                 k = 2
#             except: 
#                 print(f'ERROR on page {i+1}! {errors}')
#                 errors += 1

#         time.sleep(random.uniform(.5, 1))


    profile_urls = pd.Series(profile_urls)
    print('Done!')
    
    # Сохранение файла с URL
    profile_urls.to_csv(URL_FILE_NAME)
    print(f'URLs saved to file: {URL_FILE_NAME}')
    
    # Получение информации из профилей пользователей
    for profile_url in profile_urls:
        profile_info.loc[len(profile_info.index)] = get_and_save_profile_info(driver, profile_url)
        time.sleep(random.uniform(4, 7))
    
    # Сохранение файла с инфо пользователей
    profile_info.to_csv(INFO_FILE_NAME)
    print(f'Users info saved to file: {INFO_FILE_NAME}')

    driver.quit()

Page 1
0 https://www.linkedin.com/in/anton-morgunov-855206250?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAD33hUEBw_8tymC4A-gCRnX84mpbrottgIE
0 https://www.linkedin.com/in/daniil-kiryakov-1b7a44248?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAD1g2BsBxmXXifSq2jd_Q7stXd42EUcpvrg
0 https://www.linkedin.com/in/svit?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAB33j0UBKXEyrKc-Dwnv6pete8WoXlGPynU
0 https://www.linkedin.com/in/meffazm?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAADC40OgBcNbVsr02FzlvnKxDnPBVGZL0Stg
0 https://www.linkedin.com/in/valeriy-loev-005411249?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAD12l8UBvL0CJ3SA05hQlCfC1mPmz7-Lut4
0 https://www.linkedin.com/in/anechiporenko?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAABx0lmoBcthPwMMHEMPcl_mZQ1brUjnJXdY
0 https://www.linkedin.com/in/danila-ukader-672628127?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3AACoAAB89E7YBOnBuuIS8j-WHU1tcAM691c-deZ4
0 https://www.linkedin.com/in/nmsh?miniProfileUrn=urn%3Ali%3Afs_miniProfile%3

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=114.0.5735.134)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00A0A813+48355]
	(No symbol) [0x0099C4B1]
	(No symbol) [0x008A5358]
	(No symbol) [0x0088D293]
	(No symbol) [0x008EE37B]
	(No symbol) [0x008FC473]
	(No symbol) [0x008EA536]
	(No symbol) [0x008C82DC]
	(No symbol) [0x008C93DD]
	GetHandleVerifier [0x00C6AABD+2539405]
	GetHandleVerifier [0x00CAA78F+2800735]
	GetHandleVerifier [0x00CA456C+2775612]
	GetHandleVerifier [0x00A951E0+616112]
	(No symbol) [0x009A5F8C]
	(No symbol) [0x009A2328]
	(No symbol) [0x009A240B]
	(No symbol) [0x00994FF7]
	BaseThreadInitThunk [0x75E100C9+25]
	RtlGetAppContainerNamedObjectPath [0x77527B4E+286]
	RtlGetAppContainerNamedObjectPath [0x77527B1E+238]
